<a href="https://colab.research.google.com/github/pacomm5/PLANTILLAS2/blob/main/COHORT3_MIO_Carroceria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#AQUI VOY A HACER EL DE CARROCERÍA PARA MARKETING ESTE ES EL NUEVO. 
este es el nuevo despues de corregir. En teoría esta bien. 

In [1]:
import pandas as pd
# me quedo aquí tengo que volver a hacerlo a ver si me aclaro. 
# entiendo que al quitar los duplicados de las orden corta2, también me estoy cargando el bastidor y me va a salir mal. 

In [2]:
xlsx = pd.ExcelFile("/content/drive/MyDrive/Colab Notebooks/U239_IMPORTADO.xlsx") # ESTE EL NOMBRE DE LA RUTA. 
df = pd.read_excel(xlsx, "U239_IMPORTADO" ) # ESTE EL NOMBRE DE LA PESTAÑA. 
# he descargado el archivo en google drive para no tener que estar cargarlo cada vez que lo utilice. 

In [3]:
# for data manipulation and analysis
import pandas as pd
import numpy as np

# for plotting
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
sns.set_style('darkgrid')

# Silhouette analysis
from sklearn.metrics import silhouette_score

# To perform KMeans clustering 
from sklearn.cluster import KMeans

# for scaling
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')

In [4]:
pd.options.display.max_columns = 0 # con esto no olimitamos que se vean todas las columnas a lo ancho

In [5]:
df.columns

Index(['Tall', 'Referencia', 'Fecha Fa', 'Fec.aper', 'Cta.car',
       'Nombre cliente', 'Cta.tit', 'Nombre titular', 'Recamb.',
       'RECAMBIOS PVP', 'Tiem.fac', 'Total.MO', 'Modelo', 'Recep', 'Matricula',
       'Bastidor', 'Tipo O.R.', 'OR UNICA', 'MES', 'AÑO', 'ESTANCIA',
       'ORDEN CORTA'],
      dtype='object')

In [6]:
# voy a cambiar el nombre de algunas columnas. 
df = df.rename(columns={"Fecha Fa":"Fecha_Fact", "Fec.aper":"Fecha_Apertura", "Cta.car":"Cta_Cargo","Nombre cliente":"Nombre_Cliente", "Cta.tit":"Cta_Titular", "Nombre titular":"Nombre_Titular", 
                   "Recamb.":"Recamb", "RECAMBIOS PVP":"Recamb_Pvp", "Tiem.fac":"Tiemp_Fact", "Total.MO":"Total_Mo", "Tipo O.R.":"Tipo_Or", "OR UNICA": "Or_Unica", "ORDEN CORTA":"Orden_Corta" })


In [7]:
print("la fecha va desde {} a {}".format(df.Fecha_Fact.min(), df.Fecha_Fact.max()))

la fecha va desde 2015-01-02 00:00:00 a 2022-12-30 00:00:00


In [8]:
# lo primero que voy a hacer es cambiar de tipo de datos. 
df["Tall"] = df["Tall"].apply(str)
df["Referencia"] = df["Referencia"].apply(str)
df["Cta_Cargo"] = df["Cta_Cargo"].apply(str)
df["Or_Unica"] = df["Or_Unica"].apply(str)
df["Orden_Corta"] = df["Orden_Corta"].apply(str)
df["Cta_Cargo"] = df["Cta_Cargo"].apply(str)
df["Nombre_Cliente"] = df["Nombre_Cliente"].apply(str)
df["Cta_Titular"] = df["Cta_Titular"].apply(str)
df["Nombre_Titular"] = df["Nombre_Titular"].apply(str)
df["Modelo"] = df["Modelo"].apply(str)
df["Recep"] = df["Recep"].apply(str)
df["Matricula"] = df["Matricula"].apply(str)
df["Bastidor"] = df["Bastidor"].apply(str)
df["Tipo_Or"] = df["Tipo_Or"].apply(str)


In [9]:
df["Fact_total"] = df["Total_Mo"] + df["Recamb"]

In [10]:
# de esta manera lo que hago es quitar el último digito de la variable referencia. 
# lo he hecho para probar como se hace. podría haberlo metido en otra variable y luego meterla en el dataframe para no eliminar información. 
df["Orden_Corta2"] = df.apply(lambda x: x["Referencia"][:-1], axis=1)

In [11]:
# voy a quitar las internas. 
df = df[df["Tipo_Or"].str.startswith("5")!=True]
# voy a quitar las internas. 
df = df[df["Tipo_Or"].str.startswith("6")!=True]

In [12]:
df.head()

,Tall,Referencia,Fecha_Fact,Fecha_Apertura,Cta_Cargo,Nombre_Cliente,Cta_Titular,Nombre_Titular,Recamb,Recamb_Pvp,Tiemp_Fact,Total_Mo,Modelo,Recep,Matricula,Bastidor,Tipo_Or,Or_Unica,MES,AÑO,ESTANCIA,Orden_Corta,Fact_total,Orden_Corta2
0,6,7655783,2015-01-02,2014-11-11,116,ORDENES DE REPARACION G,130187,GARCIA VILLAYZAN JOSE E,846.25,2336.55,11.13,788.47,A4 2.0 DIE,56,5249DJV,WAUZZZ8E65A483946,4A GARANTIA MECAN AUDI,nan,1,2015,52,765578,1634.72,765578
2,6,7655785,2015-01-02,2014-11-11,610662,GASTOS ASUMIDOS POR TAL,130187,GARCIA VILLAYZAN JOSE E,644.11,2134.41,6.77,324.90,A4 2.0 DIE,56,5249DJV,WAUZZZ8E65A483946,DG DEFER.GARANTIA AUDI,nan,1,2015,52,765578,969.01,765578
3,1,7683941,2015-01-02,2014-11-24,99234,SALVADOR DEL OLMO GERMA,99234,SALVADOR DEL OLMO GERMA,1675.97,3351.94,1.10,84.70,A-4 2.0 TD,272,8087DGN,WAUZZZ8E85A473547,2A O.R.CLIENT MEC AUDI,nan,1,2015,39,768394,1760.67,768394
4,2,7702661,2015-01-02,2014-12-30,71917,RODRIGUEZ RICO MARIA CA,71917,RODRIGUEZ RICO MARIA CA,50.04,116.40,2.10,147.41,POLO 1.4 8,50,8194FBX,WVWZZZ9NZ7Y003242,2V O.R.CLIENT MEC VW,nan,1,2015,3,770266,197.45,770266
5,1,7716492,2015-01-02,2014-12-11,116,ORDENES DE REPARACION G,117453,LOPEZ NAVARRO JOSE ANTO,0.00,0.00,0.37,26.20,A4 2.0TDI,272,5848HKH,WAUZZZ8K3DA012518,4A GARANTIA MECAN AUDI,nan,1,2015,22,771649,26.20,771649


In [13]:
df_carroceria = df.copy() # abro df_carroceria como copia de de df. 

In [16]:
df_carroceria = df_carroceria[df_carroceria["Tipo_Or"].str.startswith("7")==True] # le dejo solo las ordenes que empiecen por 7

In [17]:
df_carroceria.head()

,Tall,Referencia,Fecha_Fact,Fecha_Apertura,Cta_Cargo,Nombre_Cliente,Cta_Titular,Nombre_Titular,Recamb,Recamb_Pvp,Tiemp_Fact,Total_Mo,Modelo,Recep,Matricula,Bastidor,Tipo_Or,Or_Unica,MES,AÑO,ESTANCIA,Orden_Corta,Fact_total,Orden_Corta2
64,6,7737201,2015-01-05,2014-12-22,130214,RICARDO GARNICA COLLANT,130214,RICARDO GARNICA COLLANT,5.20,10.40,7.53,594.98,AUDI A6 V6,63,M5975VX,WAUZZZ4AZSN065279,7A PRESUPUESTO CH AUDI,nan,1,2015,14,773720,600.18,773720
85,2,7572434,2015-01-07,2014-09-30,18918,. . JARMAUTO SA,120,TORRES DE LA PEÑA JOAQU,1624.52,3348.23,7.95,404.13,TIGUAN 2.0,255,1243HDV,WVGZZZ5NZBW562392,7V PRESUPUESTO CH VW,nan,1,2015,99,757243,2028.65,757243
87,1,7615971,2015-01-07,2014-11-17,98253,MUTUA MADRILEÑA AUTOMO,2222,BOVEDA DE MIGUEL ANGEL,410.99,867.64,4.87,238.94,AUDI COUPE,95,M5782XB,TRUZZZ8NZX1005460,7A PRESUPUESTO CH AUDI,nan,1,2015,51,761597,649.93,761597
88,1,7615973,2015-01-07,2014-11-17,98253,MUTUA MADRILEÑA AUTOMO,2222,BOVEDA DE MIGUEL ANGEL,0.00,0.00,2.90,143.38,AUDI COUPE,95,M5782XB,TRUZZZ8NZX1005460,7A PRESUPUESTO CH AUDI,nan,1,2015,51,761597,143.38,761597
89,2,7625961,2015-01-07,2014-10-27,6588,MAPFRE FAMILIAR S.A,34070,HERRERO TRIGO JOSE LUIS,1178.60,2357.20,15.17,794.63,GOLF TDI 1,255,1208CGF,WVWZZZ1JZ3W519157,7V PRESUPUESTO CH VW,nan,1,2015,72,762596,1973.23,762596


In [18]:
# AQUI ESTOY COMPROBANDO QUE SOLO ME HAYA QUEDADO CON CARROCERÍA. 
df_carroceria.Tipo_Or.value_counts(normalize=True) # por numero de lineas, debajo están los pesos. 

7A PRESUPUESTO CH AUDI       0.574434
7V PRESUPUESTO CH VW         0.381218
7LA PRES  AU CH  LONG DRI    0.025749
7LV PRES  VW CH  LONG DRI    0.013439
7L PRESUPUESTO CH LCV        0.004408
7LL PRES  LCV  CH  LONG D    0.000753
Name: Tipo_Or, dtype: float64

In [19]:
# antes de nada lo que voy a hacer es ver los valores null
df_carroceria.isnull().sum()

Tall                 0
Referencia           0
Fecha_Fact        1640
Fecha_Apertura       0
Cta_Cargo            0
Nombre_Cliente       0
Cta_Titular          0
Nombre_Titular       0
Recamb               0
Recamb_Pvp           0
Tiemp_Fact           0
Total_Mo             0
Modelo               0
Recep                0
Matricula            0
Bastidor             0
Tipo_Or              0
Or_Unica             0
MES                  0
AÑO                  0
ESTANCIA             0
Orden_Corta          0
Fact_total           0
Orden_Corta2         0
dtype: int64

In [20]:
df_carroceria = df_carroceria[~df_carroceria.Fecha_Fact.isnull()] # los quito

In [21]:
df_carroceria.isnull().sum() # tengo algunos pero no voy  aseguir quitando. 

Tall              0
Referencia        0
Fecha_Fact        0
Fecha_Apertura    0
Cta_Cargo         0
Nombre_Cliente    0
Cta_Titular       0
Nombre_Titular    0
Recamb            0
Recamb_Pvp        0
Tiemp_Fact        0
Total_Mo          0
Modelo            0
Recep             0
Matricula         0
Bastidor          0
Tipo_Or           0
Or_Unica          0
MES               0
AÑO               0
ESTANCIA          0
Orden_Corta       0
Fact_total        0
Orden_Corta2      0
dtype: int64

In [22]:
df_carroceria.head()

,Tall,Referencia,Fecha_Fact,Fecha_Apertura,Cta_Cargo,Nombre_Cliente,Cta_Titular,Nombre_Titular,Recamb,Recamb_Pvp,Tiemp_Fact,Total_Mo,Modelo,Recep,Matricula,Bastidor,Tipo_Or,Or_Unica,MES,AÑO,ESTANCIA,Orden_Corta,Fact_total,Orden_Corta2
64,6,7737201,2015-01-05,2014-12-22,130214,RICARDO GARNICA COLLANT,130214,RICARDO GARNICA COLLANT,5.20,10.40,7.53,594.98,AUDI A6 V6,63,M5975VX,WAUZZZ4AZSN065279,7A PRESUPUESTO CH AUDI,nan,1,2015,14,773720,600.18,773720
85,2,7572434,2015-01-07,2014-09-30,18918,. . JARMAUTO SA,120,TORRES DE LA PEÑA JOAQU,1624.52,3348.23,7.95,404.13,TIGUAN 2.0,255,1243HDV,WVGZZZ5NZBW562392,7V PRESUPUESTO CH VW,nan,1,2015,99,757243,2028.65,757243
87,1,7615971,2015-01-07,2014-11-17,98253,MUTUA MADRILEÑA AUTOMO,2222,BOVEDA DE MIGUEL ANGEL,410.99,867.64,4.87,238.94,AUDI COUPE,95,M5782XB,TRUZZZ8NZX1005460,7A PRESUPUESTO CH AUDI,nan,1,2015,51,761597,649.93,761597
88,1,7615973,2015-01-07,2014-11-17,98253,MUTUA MADRILEÑA AUTOMO,2222,BOVEDA DE MIGUEL ANGEL,0.00,0.00,2.90,143.38,AUDI COUPE,95,M5782XB,TRUZZZ8NZX1005460,7A PRESUPUESTO CH AUDI,nan,1,2015,51,761597,143.38,761597
89,2,7625961,2015-01-07,2014-10-27,6588,MAPFRE FAMILIAR S.A,34070,HERRERO TRIGO JOSE LUIS,1178.60,2357.20,15.17,794.63,GOLF TDI 1,255,1208CGF,WVWZZZ1JZ3W519157,7V PRESUPUESTO CH VW,nan,1,2015,72,762596,1973.23,762596


In [24]:
# voy a comprobar si hay duplicados en orden corta 2
num_duplicados_ordencorta2 = df_carroceria["Orden_Corta2"].duplicated().sum() # con esto sabemos cuantos duplicados hay en la columna. 
num_duplicados_ordencorta2 # y como vemos hay muchos duplicados. 
# por lo tanto lo primero que vamos a hacer es quitar los duplicados para quitar el número de visitas. 

19419

In [25]:
# primero hago una copia del df_carrocería para no perderlo. 
df_carroceria_sinduplicados_orcorta2 = df_carroceria.copy() # creo un nuevo dataframe df_carroceria_sinduplicados_orcorta2

In [26]:
df_carroceria_sinduplicados_orcorta2.head()

,Tall,Referencia,Fecha_Fact,Fecha_Apertura,Cta_Cargo,Nombre_Cliente,Cta_Titular,Nombre_Titular,Recamb,Recamb_Pvp,Tiemp_Fact,Total_Mo,Modelo,Recep,Matricula,Bastidor,Tipo_Or,Or_Unica,MES,AÑO,ESTANCIA,Orden_Corta,Fact_total,Orden_Corta2
64,6,7737201,2015-01-05,2014-12-22,130214,RICARDO GARNICA COLLANT,130214,RICARDO GARNICA COLLANT,5.20,10.40,7.53,594.98,AUDI A6 V6,63,M5975VX,WAUZZZ4AZSN065279,7A PRESUPUESTO CH AUDI,nan,1,2015,14,773720,600.18,773720
85,2,7572434,2015-01-07,2014-09-30,18918,. . JARMAUTO SA,120,TORRES DE LA PEÑA JOAQU,1624.52,3348.23,7.95,404.13,TIGUAN 2.0,255,1243HDV,WVGZZZ5NZBW562392,7V PRESUPUESTO CH VW,nan,1,2015,99,757243,2028.65,757243
87,1,7615971,2015-01-07,2014-11-17,98253,MUTUA MADRILEÑA AUTOMO,2222,BOVEDA DE MIGUEL ANGEL,410.99,867.64,4.87,238.94,AUDI COUPE,95,M5782XB,TRUZZZ8NZX1005460,7A PRESUPUESTO CH AUDI,nan,1,2015,51,761597,649.93,761597
88,1,7615973,2015-01-07,2014-11-17,98253,MUTUA MADRILEÑA AUTOMO,2222,BOVEDA DE MIGUEL ANGEL,0.00,0.00,2.90,143.38,AUDI COUPE,95,M5782XB,TRUZZZ8NZX1005460,7A PRESUPUESTO CH AUDI,nan,1,2015,51,761597,143.38,761597
89,2,7625961,2015-01-07,2014-10-27,6588,MAPFRE FAMILIAR S.A,34070,HERRERO TRIGO JOSE LUIS,1178.60,2357.20,15.17,794.63,GOLF TDI 1,255,1208CGF,WVWZZZ1JZ3W519157,7V PRESUPUESTO CH VW,nan,1,2015,72,762596,1973.23,762596


In [27]:
# ahora lo que hago es ordenarlos por fecha de Facturación en orden descedente y me quedo con la primera, que es la fecha más reciente. 
df_carroceria_sinduplicados_orcorta2 = df_carroceria_sinduplicados_orcorta2.sort_values(by="Fecha_Fact", ascending=False).drop_duplicates(
    subset=["Orden_Corta2"], keep= "first")

In [28]:
# voy a comprobar si tengo duplicados después de hacerlo.
num_duplicados = df_carroceria_sinduplicados_orcorta2["Orden_Corta2"].duplicated().sum()
num_duplicados
# y vemos que no tenemos ningún duplicado. 

0

In [38]:
# voy a comprobar si tengo duplicados después de hacerlo.
num_duplicados2 = df_carroceria_sinduplicados_orcorta2["Bastidor"].duplicated().sum()
num_duplicados2
# evidentemente si tengo duplicados de bastidores. 

14538

In [33]:
# ahora sí puedo calcular el numero de visitas. 
visitas = pd.DataFrame(df_carroceria_sinduplicados_orcorta2.groupby("Bastidor")["Orden_Corta2"].nunique().reset_index())
visitas.columns = ["vBastidor", "visitas"]
visitas.head()

,vBastidor,visitas
0,WAUZZZ8E55A50910,1
1,WAUZZZ8P07A02124,2
2,WAUZZZ8T4EA06800,1
3,1C4BU0000HPF56406,1
4,1JBG2E8A84Y142079,1


In [ ]:
visitas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51759 entries, 0 to 51758
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   vBastidor  51759 non-null  object
 1   visitas    51759 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 808.9+ KB


In [34]:
ultima_visita = pd.DataFrame(df_carroceria.groupby("Bastidor")["Fecha_Fact"].max().reset_index())
ultima_visita.columns = ["uBastidor", "Ulti_visita"]
ultima_visita.head()


,uBastidor,Ulti_visita
0,WAUZZZ8E55A50910,2017-12-20
1,WAUZZZ8P07A02124,2021-09-06
2,WAUZZZ8T4EA06800,2017-07-31
3,1C4BU0000HPF56406,2022-01-07
4,1JBG2E8A84Y142079,2020-01-27


In [35]:
ultima_visita.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20211 entries, 0 to 20210
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   uBastidor    20211 non-null  object        
 1   Ulti_visita  20211 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 315.9+ KB


In [36]:
df_carroceria_marketing = pd.concat([visitas, ultima_visita], axis=1) # lo que hago es concatenar los dos dataframe que he calculado. 
# pongo axis 1 para que la concatenación se haga en horizontal. si pusiera 0 se haría uno debajo de otro. 
# Tienen que tener las mismas filas para podere hacer lo primero. No estoy concatenando por ningún ID. 
df_carroceria_marketing.drop([ "uBastidor"], axis=1, inplace=True)
df_carroceria_marketing.head()

,vBastidor,visitas,Ulti_visita
0,WAUZZZ8E55A50910,1,2017-12-20
1,WAUZZZ8P07A02124,2,2021-09-06
2,WAUZZZ8T4EA06800,1,2017-07-31
3,1C4BU0000HPF56406,1,2022-01-07
4,1JBG2E8A84Y142079,1,2020-01-27


In [37]:
df_carroceria_marketing.to_excel("/content/drive/MyDrive/Colab Notebooks/Carroceria_unicos.xlsx", index=False)